In [1]:
import pickle

import tqdm
import numpy as np
from PIL import Image

In [2]:
def load(filename):
    """Load MNIST images
    Returns:
      4 matrices : 'training_images', 'training_labels', 'test_images', 'test_labels'
    """
    with open(filename,'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

def preproccess_images(X, Y, split):
    """Convert to binary format image, extracts only 0 and 1
    classes from train dataset.
    """
    # convert image from range {0,1,...,255} to range {0,1}
    X = (X>127).astype(np.uint8)
    # get indecies of images, which represent 0 and 1
    indecies_of_zeros = np.where(Y == 0)
    indecies_of_ones = np.where(Y == 1)
    # amount of 0 and 1 digits in 'X_train'
    number_zeros = len(indecies_of_zeros[0])
    number_ones = len(indecies_of_ones[0])
    # get 1 and 0 digits from 'X_train'
    zeros = X[indecies_of_zeros,:].reshape((-1,28,28))
    ones = X[indecies_of_ones,:].reshape((-1,28,28))
    n_samples = min(number_ones,number_zeros)
    train_numbers = int(split*n_samples)
    zeros_train = zeros[:train_numbers,...]
    zeros_test = zeros[train_numbers:n_samples,...]
    ones_train = ones[:train_numbers,...]
    ones_test = ones[train_numbers:n_samples,...]
    X_train = np.concatenate([zeros_train,ones_train], axis=0)
    Y_train = np.concatenate((np.zeros((train_numbers,1)),np.ones((train_numbers,1))), axis=0).astype(np.uint8)    
    return X_train, Y_train

######

def generate_aposterior_probabilities(n_samples,n_classes):
    """Generate a matrix with shape (n_samples,n_classes), where
    sum along axis=1 = 1. p(k|Xz)
    """
    rand_matrix = np.random.randint(low=0, high=5000,size=(n_samples,n_classes))
    sum_along_row = np.repeat(rand_matrix.sum(axis=1).reshape((-1,1)),repeats=2, axis=1)
    prob_matrix = rand_matrix/sum_along_row
    return prob_matrix


def aprior_probabilities(probability_histogram):
    """calculate p(k)
    """
    return probability_histogram.mean(axis=0)



def calculate_parameters(X, aposterior_probs):
    """Calculate p_kij
    Args:
        X : a matrix (8982,28,28)
        aposterior_probs : a matrix (8982,2)
    Returns :
    a matrix of probabilities
    """
    znam = aposterior_probs.sum(axis=0)
    znam = np.expand_dims(znam,axis=1)
    znam = np.repeat(znam,repeats=28*28, axis=1).reshape((2,28,28))
    probs = np.expand_dims(aposterior_probs.T,axis=2)
    probs = np.repeat(probs,repeats=28*28, axis=2).reshape((2,X.shape[0],28,28))
    X = np.expand_dims(X,axis=0)
    X = np.repeat(X,repeats=2,axis=0)
    return np.multiply(X,probs).sum(axis=1)/znam


def calculate_conditional_probs(X, parameters):
    """ p_kij
    X - (8982,28,28)
    parameters - (2,28,28)
    
    Returns : 
    a matrix of shape (8292,2)
    """
    parameters = np.expand_dims(parameters, axis=0)
    parameters = np.repeat(parameters,repeats=X.shape[0], axis=0)
    parameters = parameters.reshape((parameters.shape[0],parameters.shape[1],-1))
    X = np.expand_dims(X, axis=1)
    X = np.repeat(X,repeats=2, axis=1)
    X = X.reshape((X.shape[0],X.shape[1],-1))
    power1 = np.power(parameters,X).prod(axis=-1)
    power2 = np.power(1 - parameters,(1 - X)).prod(axis=-1)
    return np.multiply(power1, power2)


def calculate_aposterior(conditions, aprior_probs):
    """
    conditions - (8982,2)
    aprior_probs - (2,)
    Returns :
    a matrix with shape (8292,2)
    """
    znam = conditions[:,0]*aprior_probs[0] + conditions[:,1]*aprior_probs[1]
    aposterior0 = (conditions[:,0]*aprior_probs[0])/znam
    aposterior1 = (conditions[:,1]*aprior_probs[1])/znam
    return np.stack((aposterior0,aposterior1),axis=1)
    

### load mnist dataset

In [3]:
TRAIN_TEST_SPLIT = 0.7
filename = "./mnist.pkl"
num_classes = 2
epochs = 20
##############
# in the further parts of lab we use only train part of MNIST dataset 
X, Y, _, _ = load(filename)
# convert to binary format, extract only 0 and 1 classes
X_train, Y_train = preproccess_images(X,Y, TRAIN_TEST_SPLIT)
print("X_train.shape = ",X_train.shape)
# example 
Image.fromarray(X_train[234,:,:]*255)

X_train.shape =  (8292, 28, 28)


In [4]:
for epoch in tqdm.tqdm(range(epochs)):
    if epoch == 0 :
        # p(k|Xz), shape=(8982,2)
        aposterior_probs = generate_aposterior_probabilities(X_train.shape[0], num_classes)
        print("aposterior_probs.shape = ",aposterior_probs.shape)
    # p(k), shape = (2,)
    aprior_probs = aprior_probabilities(aposterior_probs)
    # p_kij, shape= (2,28,28)
    parameters = calculate_parameters(X_train,aposterior_probs)
    # p(Xz|k), shape=(8982,2)
    conditions = calculate_conditional_probs(X_train, parameters)
    # p(k|Xz), shape=(8982,2)
    aposterior_probs = calculate_aposterior(conditions,aprior_probs)

  0%|          | 0/20 [00:00<?, ?it/s]

aposterior_probs.shape =  (8292, 2)


100%|██████████| 20/20 [00:41<00:00,  2.06s/it]


In [5]:
Image.fromarray((parameters[0,:,:]*255).astype(np.uint8))

In [6]:
Image.fromarray((parameters[1,:,:]*255).astype(np.uint8))